In [1]:
import numpy as np
import os, glob
from skimage import io, exposure, img_as_ubyte
import xtiff

In [2]:
def write_composite(data_path, plateID, pipeline_name, well, field_num):
    c_filename = os.path.join(data_path, plateID,"Analysis",pipeline_name,"intermediate_files",plateID+"_C_"+well+"_"+str(field_num)+"_reg_stack.ome.tiff")
    print("creating composite movie file for "+c_filename)
    r_src = os.path.join(data_path, plateID,"Analysis",pipeline_name,"intermediate_files",plateID+"_R_"+well+"_"+str(field_num)+"_reg_stack.tif")
    g_src = r_src.replace("_R_","_G_")
    p_src = r_src.replace("_R_","_P_")

    mask_src = os.path.join(data_path, plateID,"Analysis",pipeline_name,"intermediate_files/tracking",well,"field_"+str(field_num),"filtered_masks")
    mask_filenames = sorted(glob.glob(mask_src+"/mask*"))
    r_raw = io.imread(r_src)
    g_raw = io.imread(g_src)
    p_raw = io.imread(p_src)
    r_stack = img_as_ubyte(exposure.rescale_intensity(r_raw, in_range = (np.quantile(r_raw, .07),np.quantile(r_raw, .9995))))
    g_stack = img_as_ubyte(exposure.rescale_intensity(g_raw, in_range = (np.quantile(g_raw, .05),np.quantile(g_raw, .9995))))
    p_stack = img_as_ubyte(exposure.rescale_intensity(p_raw))
    
    mask_list = []
    for fn in mask_filenames:
        #read in the mask image
        masks = io.imread(fn)
        mask_list.append(masks)
    mask_stack = img_as_ubyte(exposure.rescale_intensity(np.stack(mask_list)))
    composite_stack = np.stack((r_stack, g_stack, mask_stack, p_stack), axis = 1) #TCYX
    res = xtiff.to_tiff(composite_stack, c_filename)
    return(composite_stack)


In [3]:
data_path = "/home/exacloud/gscratch/HeiserLab/images/"
plateID_list = ("2100701", "2100801")
#plateID_list = (["2100701"])
pipeline_name = "CKn"
well_list = ("A1", "D2", "D4", "D6")
#well_list = (["D2"])

field_num = 1

res = list(map(write_composite, [data_path]*len(well_list)*len(plateID_list), [element for element in plateID_list for i in range(len(well_list))], [pipeline_name]*len(well_list)*len(plateID_list), well_list*len(plateID_list), [field_num]*len(well_list)*len(plateID_list)))

creating composite movie file for /home/exacloud/gscratch/HeiserLab/images/2100701/Analysis/CKn/intermediate_files/2100701_C_A1_1_reg_stack.ome.tiff
creating composite movie file for /home/exacloud/gscratch/HeiserLab/images/2100701/Analysis/CKn/intermediate_files/2100701_C_D2_1_reg_stack.ome.tiff
creating composite movie file for /home/exacloud/gscratch/HeiserLab/images/2100701/Analysis/CKn/intermediate_files/2100701_C_D4_1_reg_stack.ome.tiff
creating composite movie file for /home/exacloud/gscratch/HeiserLab/images/2100701/Analysis/CKn/intermediate_files/2100701_C_D6_1_reg_stack.ome.tiff
creating composite movie file for /home/exacloud/gscratch/HeiserLab/images/2100801/Analysis/CKn/intermediate_files/2100801_C_A1_1_reg_stack.ome.tiff
creating composite movie file for /home/exacloud/gscratch/HeiserLab/images/2100801/Analysis/CKn/intermediate_files/2100801_C_D2_1_reg_stack.ome.tiff
creating composite movie file for /home/exacloud/gscratch/HeiserLab/images/2100801/Analysis/CKn/intermedia